**Exemplo 01: Transformação de Helmert 2D**

A transformação de Helmert 2D é utilizada para converter coordenadas de um sistema para outro, considerando translação, rotação e escala. Vamos ajustar os parâmetros dessa transformação a partir de um conjunto de pontos com coordenadas conhecidas em ambos os sistemas.

**1. Dados de Entrada:**

| Ponto | X (Sistema 1) | Y (Sistema 1) | X (Sistema 2) | Y (Sistema 2) |
|---|---|---|---|---|
| 1 | 1000.00 | 2000.00 | 1005.25 | 1998.50 |
| 2 | 2500.00 | 3000.00 | 2506.75 | 2997.25 |
| 3 | 4000.00 | 1500.00 | 4008.00 | 1496.00 |

**2. Modelo Matemático:**

A transformação de Helmert 2D é definida por:

```
X2 = TX + (1 + S) * (X1 * cos(R) - Y1 * sin(R))
Y2 = TY + (1 + S) * (X1 * sin(R) + Y1 * cos(R))
```

Onde:

* X1, Y1: Coordenadas no sistema 1
* X2, Y2: Coordenadas no sistema 2
* TX, TY: Parâmetros de translação
* R: Parâmetro de rotação
* S: Parâmetro de escala

**3. Implementação em Python (usando a biblioteca SciPy):**

In [1]:
import numpy as np
from scipy.optimize import least_squares

# Dados de entrada
X1 = np.array([1000.00, 2500.00, 4000.00])
Y1 = np.array([2000.00, 3000.00, 1500.00])
X2 = np.array([1005.25, 2506.75, 4008.00])
Y2 = np.array([1998.50, 2997.25, 1496.00])

# Valores iniciais para os parâmetros
TX_0, TY_0, R_0, S_0 = 0, 0, 0, 0

# Função para calcular os resíduos
def residuos(params, X1, Y1, X2, Y2):
    TX, TY, R, S = params
    res_X = TX + (1 + S) * (X1 * np.cos(R) - Y1 * np.sin(R)) - X2
    res_Y = TY + (1 + S) * (X1 * np.sin(R) + Y1 * np.cos(R)) - Y2
    return np.concatenate((res_X, res_Y))

# Ajustamento
resultado = least_squares(residuos, [TX_0, TY_0, R_0, S_0], args=(X1, Y1, X2, Y2))

# Parâmetros ajustados
TX, TY, R, S = resultado.x
print("Parâmetros ajustados (TX, TY, R, S):", TX, TY, np.degrees(R), S)

Parâmetros ajustados (TX, TY, R, S): 3.3602954678547317 -3.1691176487402437 -0.031991397026070505 0.0008383910427413169


**4. Explicação do Código:**

* **Dados de Entrada:** Define as coordenadas dos pontos em ambos os sistemas.
* **Valores Iniciais:** Define estimativas iniciais para os parâmetros da transformação.
* **Função `residuos`:** Calcula os resíduos para as coordenadas X e Y.
* **Ajustamento:** Utiliza a função `least_squares` para encontrar os parâmetros que minimizam a soma dos quadrados dos resíduos.

**5. Resultados:**

O código fornecerá os parâmetros ajustados da transformação de Helmert 2D (TX, TY, R, S), que podem ser utilizados para converter coordenadas entre os dois sistemas com maior precisão.

**Observações:**

* A transformação de Helmert 2D é um caso particular de transformação conforme, que preserva ângulos.
* É possível adicionar pesos às observações para considerar diferentes precisões.
* A análise dos resíduos e da matriz de variância-covariância dos parâmetros ajustados permite avaliar a qualidade do ajustamento.

**Exemplo 02: Ajustamento de uma Rede de Triangulação**

Consideremos uma rede de triangulação com três pontos (A, B e C), cujas coordenadas são conhecidas, e um ponto novo (P) cujas coordenadas desejamos determinar. Foram realizadas as seguintes medições:

* Distâncias:
    * dAP = 1500.25 m
    * dBP = 1200.50 m
    * dCP = 1800.75 m
* Ângulos:
    * αABP = 45° 10' 20"
    * αBCP = 60° 30' 40"

**1. Implementação em Python (usando a biblioteca SciPy):**

In [ ]:
import numpy as np
from scipy.optimize import least_squares

# Dados de entrada (observações)
L = np.array([1500.25, 1200.50, 1800.75, np.radians(45 + 10/60 + 20/3600), np.radians(60 + 30/60 + 40/3600)])
P = np.diag([1, 1, 1, 1, 1])  # Matriz peso (considerando igual precisão)

# Coordenadas dos pontos conhecidos
XA, YA = 1000, 1000
XB, YB = 2000, 1500
XC, YC = 3000, 1000

# Valores iniciais para as coordenadas de P
XP_0, YP_0 = 2500, 1200

# Função para calcular os resíduos
def residuos(X, L, XA, YA, XB, YB, XC, YC):
    XP, YP = X
    res = np.zeros(5)
    res[0] = np.sqrt((XP - XA)**2 + (YP - YA)**2) - L[0]
    res[1] = np.sqrt((XP - XB)**2 + (YP - YB)**2) - L[1]
    res[2] = np.sqrt((XP - XC)**2 + (YP - YC)**2) - L[2]
    res[3] = np.arctan2(YP - YA, XP - XA) - np.arctan2(YB - YA, XB - XA) - L[3]
    res[4] = np.arctan2(YP - YC, XP - XC) - np.arctan2(YB - YC, XB - YC) - L[4]
    return res

# Ajustamento pelo método combinado
resultado = least_squares(residuos, [XP_0, YP_0], args=(L, XA, YA, XB, YB, XC, YC), method='trf')

# Coordenadas ajustadas de P
XP, YP = resultado.x
print("Coordenadas ajustadas de P (XP, YP):", XP, YP)

**2. Explicação do Código:**

* **Dados de Entrada:** Define as observações (distâncias e ângulos) e a matriz peso.
* **Coordenadas Conhecidas:** Define as coordenadas dos pontos A, B e C.
* **Valores Iniciais:** Define estimativas iniciais para as coordenadas de P.
* **Função `residuos`:** Calcula os resíduos (diferenças entre valores observados e calculados) para cada observação.
* **Ajustamento:** Utiliza a função `least_squares` da biblioteca SciPy para minimizar a soma dos quadrados dos resíduos, encontrando as coordenadas ajustadas de P.

**3. Resultados:**

O código fornecerá as coordenadas ajustadas do ponto P (XP, YP), que representam a solução do problema de ajustamento pelo método combinado.

**Observações:**

* O método combinado é versátil e pode ser aplicado a diversos problemas de ajustamento em geodésia.
* A qualidade do ajustamento pode ser avaliada pela análise dos resíduos e da matriz de variância-covariância dos parâmetros ajustados.

**Exemplo 03: Transformação de Helmert 7 Parâmetros (3D)**

A transformação de Helmert 7 parâmetros é uma extensão da transformação 2D, incluindo um parâmetro de escala para cada eixo e três parâmetros de rotação (em torno dos eixos X, Y e Z).

**1. Dados de Entrada:**

Vamos considerar um conjunto de 10 pontos com coordenadas conhecidas em dois sistemas de referência (Sistema 1 e Sistema 2):

| Ponto | X (Sistema 1) | Y (Sistema 1) | Z (Sistema 1) | X (Sistema 2) | Y (Sistema 2) | Z (Sistema 2) |
|---|---|---|---|---|---|---|
| 1 | 1000.00 | 2000.00 | 500.00 | 1005.25 | 1998.50 | 501.50 |
| 2 | 2500.00 | 3000.00 | 800.00 | 2506.75 | 2997.25 | 802.00 |
| 3 | 4000.00 | 1500.00 | 300.00 | 4008.00 | 1496.00 | 301.00 |
| ... | ... | ... | ... | ... | ... | ... |
| 10 | 5500.00 | 4500.00 | 1200.00 | 5509.50 | 4495.75 | 1203.00 |

**2. Modelo Matemático:**

A transformação de Helmert 7 parâmetros é definida por:

```
X2 = TX + (1 + SX) * (X1 * cos(RZ) - Y1 * sin(RZ))
Y2 = TY + (1 + SY) * (X1 * sin(RZ) + Y1 * cos(RZ))
Z2 = TZ + (1 + SZ) * Z1 + X1 * (-RX * sin(RZ) - RY * cos(RZ)) + Y1 * (RX * cos(RZ) - RY * sin(RZ))
```

Onde:

* X1, Y1, Z1: Coordenadas no sistema 1
* X2, Y2, Z2: Coordenadas no sistema 2
* TX, TY, TZ: Parâmetros de translação
* RX, RY, RZ: Parâmetros de rotação (em radianos)
* SX, SY, SZ: Parâmetros de escala

**3. Implementação em Python (usando a biblioteca SciPy):**

In [ ]:
import numpy as np
from scipy.optimize import least_squares

# Dados de entrada (substitua pelos seus dados)
X1 = np.array([...])
Y1 = np.array([...])
Z1 = np.array([...])
X2 = np.array([...])
Y2 = np.array([...])
Z2 = np.array([...])

# Valores iniciais para os parâmetros
TX_0, TY_0, TZ_0, RX_0, RY_0, RZ_0, SX_0, SY_0, SZ_0 = 0, 0, 0, 0, 0, 0, 0, 0, 0

# Função para calcular os resíduos
def residuos(params, X1, Y1, Z1, X2, Y2, Z2):
    TX, TY, TZ, RX, RY, RZ, SX, SY, SZ = params
    res_X = TX + (1 + SX) * (X1 * np.cos(RZ) - Y1 * np.sin(RZ)) - X2
    res_Y = TY + (1 + SY) * (X1 * np.sin(RZ) + Y1 * np.cos(RZ)) - Y2
    res_Z = TZ + (1 + SZ) * Z1 + X1 * (-RX * np.sin(RZ) - RY * np.cos(RZ)) + Y1 * (RX * cos(RZ) - RY * sin(RZ))  - Z2
    return np.concatenate((res_X, res_Y, res_Z))

# Ajustamento
resultado = least_squares(residuos, [TX_0, TY_0, TZ_0, RX_0, RY_0, RZ_0, SX_0, SY_0, SZ_0], args=(X1, Y1, Z1, X2, Y2, Z2))

# Parâmetros ajustados
TX, TY, TZ, RX, RY, RZ, SX, SY, SZ = resultado.x
print("Parâmetros ajustados (TX, TY, TZ, RX, RY, RZ, SX, SY, SZ):", TX, TY, TZ, np.degrees(RX), np.degrees(RY), np.degrees(RZ), SX, SY, SZ)

2. Explicação do Código:

Dados de Entrada: Define as observações (distâncias e ângulos) e a matriz peso.
Coordenadas Conhecidas: Define as coordenadas dos pontos A, B e C.
Valores Iniciais: Define estimativas iniciais para as coordenadas de P.
Função residuos: Calcula os resíduos (diferenças entre valores observados e calculados) para cada observação.
Ajustamento: Utiliza a função least_squares da biblioteca SciPy para minimizar a soma dos quadrados dos resíduos, encontrando as coordenadas ajustadas de P.
3. Resultados:

O código fornecerá as coordenadas ajustadas do ponto P (XP, YP), que representam a solução do problema de ajustamento pelo método combinado.

Para testar o código aplique o seguinte conjunto de dados:
Sistema 1 (X1, Y1, Z1):

1: 2872.70, 1102.92, 1111.85
2: 5753.57, 5849.55, 639.49
3: 4659.97, 5162.21, 792.14
4: 3993.29, 2061.70, 866.36
5: 1780.09, 1909.12, 956.07
6: 1779.97, 1917.02, 1285.18
7: 1290.42, 2521.21, 699.67
8: 5330.88, 3623.78, 1014.23
9: 4005.58, 3159.73, 1092.41
10: 4540.36, 2456.15, 546.45

Sistema 2 (X2, Y2, Z2):

1: 2877.53, 1100.40, 1114.77
2: 5757.61, 5847.42, 645.58
3: 4664.11, 5159.91, 797.41
4: 3997.97, 2059.35, 870.18
5: 1784.77, 1906.40, 958.70
6: 1784.67, 1914.30, 1287.81
7: 1294.99, 2518.38, 702.24
8: 5335.29, 3621.64, 1019.33
9: 4010.07, 3157.35, 1096.66
10: 4544.97, 2453.88, 550.71

Observações:

O método combinado é versátil e pode ser aplicado a diversos problemas de ajustamento em geodésia.
A qualidade do ajustamento pode ser avaliada pela análise dos resíduos e da matriz de variância-covariância dos parâmetros ajustados.